In [2]:
import matplotlib.pyplot as plt
import matplotlib

%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2


import numpy as np
#%matplotlib inline
import os

import os

from utils_behavior import ProcessSession
import binarize2pcalcium

Autosaving every 180 seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
#################################################
#################################################
# Load upphase data
from scipy.stats import ks_2samp

#
def match_rewards_to_rois(root_dir,
                          animal_id,
                          save_flag
                          ):
    plotting = True

    plt.figure(figsize=(25,7.5))
    plt.tight_layout()

    # Get the current figure
    fig = plt.gcf()

    # Get the bounding box of the plot area
    bbox = fig.get_tightbbox(plt.gcf().canvas.get_renderer())

    # Set the figure size to match the tight bounding box
    fig.set_size_inches(bbox.width, bbox.height)



    # load yaml file
    yaml_file = os.path.join(root_dir, animal_id, animal_id+'.yaml')
    # load yaml file for the animal
    import yaml
    with open(yaml_file) as file:
        doc = yaml.load(file, Loader=yaml.FullLoader)

    #
    try:
        session_ids = np.array(doc['session_names'],dtype='str')
    except:
        session_ids = np.array(doc['session_ids'],dtype='str')

    # load roi binarization
    early0 = []
    late0 = []
    early1 = []
    late1 = []
    for ctrs, session_id in enumerate(session_ids[1:]):
        try:
            fname = os.path.join(root_dir, animal_id, session_id, 'results', 'binarized.npz')
            d = np.load(fname, allow_pickle=True)
        except:
            continue

        F_upphase = d['F_upphase']
        F_detrended = d['F_raw']

        #load reward times
        fname = os.path.join(root_dir, animal_id, session_id, 'results.npz')
        dd = np.load(fname, allow_pickle=True)
        reward_times = dd['reward_times'].T[:,1]
        
        # find the first -1 value in the reward_times
        idx = np.where(reward_times==-1)[0][0]
        reward_times = reward_times[:idx]

        # cutoff reward times to match max length of F_upphase
        rec_len_frames = 30*60*30
        idx = np.where(reward_times<rec_len_frames)[0]
        reward_times = reward_times[idx]
        n_rewards = reward_times.shape[0]

        # detect which upphases are associated with reward times
        match_array = np.zeros((reward_times.shape[0], 2), 'float32')+0.0000001
        
        # also make an array that can hold 30mins of 5min intervals
        match_array_5mins = np.zeros((6, 2), 'float32')+0.0000001

        #
        for k in range(reward_times.shape[0]):
            temp = reward_times[k]
            #print ("time of reward in 5mins chunks: ", int(temp/(30*300)))
            found_match = False
            for p in range(2):
                # check if any of the last 2 timesteps are upphases
                if F_upphase[p,temp-1:temp+1].sum()>0:
                    match_array[k,p] = 1
                    found_match = True

                    # convert the time of the reward to 5min intervals
                    # reward time is at 30hz
                    match_array_5mins[int(temp/(30*300)),p] += 1
                   # print ('temp,k,p,', temp,k,p,match_array_5mins)
            
            # if didn't find a match, take the largest value of the raw data
            if found_match==False:
                val1 = np.max(F_detrended[0,temp-1:temp+1])
                val2 = np.max(F_detrended[1,temp-1:temp+1])
                if val1>val2:
                    match_array[k,0] = 1
                    match_array_5mins[int(temp/(30*300)),0] +=1
                else:
                    match_array[k,1] = 1
                    match_array_5mins[int(temp/(30*300)),1] +=1

        # check match array and replace nans with 0
        idx = np.where(np.isnan(match_array))
        match_array[idx] = 0

        # same for match_array_5mins
        idx = np.where(np.isnan(match_array_5mins))
        match_array_5mins[idx] = 0

        # 
        temp1 = np.sum(match_array[:,0], axis=0)
        temp2 = np.sum(match_array[:,1], axis=0)
        sums = np.sum(match_array, axis=1)
        idx = np.where(sums==2)

        # normalize match_array_5mins to add up to 1 by 0'th axis
        temp = np.sum(match_array_5mins, axis=1)
        temp = np.tile(temp, (2,1)).T
        match_array_5mins_norm = match_array_5mins.copy()/temp
        #print ("match_array_5mins: ", match_array_5mins_norm)
        
        # we want to check statistical signficant between first 2 sessions and last 2 sessions of the match_array_5mins
        if ctrs<1:
            early0.append(match_array_5mins_norm[:,0])
            early1.append(match_array_5mins_norm[:,1])
        elif ctrs>5:
            late0.append(match_array_5mins_norm[:,0])
            late1.append(match_array_5mins_norm[:,1])

        # make a histogram that shows # of matches to each F_upphase, both and none
        if plotting: 
            #
            #n_rewards = 1
            ax=plt.subplot(1,3,1)
            plt.title("% rewards")
            plt.bar(ctrs*3, temp1/n_rewards, 0.9, color='blue', label = 'roi 1',alpha=.5)
            plt.bar(ctrs*3+1, temp2/n_rewards, 0.9, color='red', label = 'roi 2',alpha=.5)

            # plot also the distributiosn from match_array_5mins
            plt.scatter(ctrs*3 + match_array_5mins_norm[:,0]*0
                        # add a bit of x jitter
                        + (np.random.rand(6)-0.5)/2,
                        match_array_5mins_norm[:,0], 
                        color='blue', 
                        label = 'roi 1',
                        edgecolors='black',
                        s=100)
            plt.scatter(ctrs*3 + match_array_5mins_norm[:,1]*0+1
                        # add a bit of x jitter
                        + (np.random.rand(6)-0.5)/2,
                        match_array_5mins_norm[:,1], 
                        color='red', 
                        label = 'roi 2',
                        edgecolors='black',
                        s=100)

            # relabel x axis to go from 0 to len(session_ids[1:])
            plt.xticks(np.arange(0, len(session_ids[1:]))*3, session_ids[1:], rotation=25)

            ax=plt.subplot(1,3,2)
            plt.title(" absolute # rewards")
            plt.bar(ctrs*3, temp1/6, 0.9, color='blue', label = 'roi 1', alpha=.5)
            plt.bar(ctrs*3+1, temp2/6, 0.9, color='red', label = 'roi 2', alpha=.5)

            #
            plt.scatter(ctrs*3 + match_array_5mins[:,0]*0 
                        # add a bit of x jitter
                        + (np.random.rand(6)-0.5)/2,
                        match_array_5mins[:,0], 
                        color='blue', 
                        label = 'roi 1',
                        edgecolors='black',
                        s=100)
            plt.scatter(ctrs*3 + match_array_5mins[:,1]*0+1
                        # add a bit of x jitter
                        + (np.random.rand(6)-0.5)/2,
                        match_array_5mins[:,1], 
                        color='red', 
                        label = 'roi 2',
                        edgecolors='black',
                        s=100)

            #
            plt.xticks(np.arange(0, len(session_ids[1:]))*3, session_ids[1:], rotation=25)

            ax=plt.subplot(1,3,3)
            plt.title("# coactivations ")
            plt.bar(ctrs, idx[0].shape[0], color='green', label = 'both')
            plt.xticks(np.arange(0, len(session_ids[1:])), session_ids[1:], rotation=25)

    #
    plt.suptitle(animal_id)
    plt.savefig('/home/cat/'+ animal_id+'_rewards_vs_rois.png')
    plt.show()
    plt.close()

    ##################################################
    # make a figure just for early0 and late0
    plt.figure(figsize=(15,7.5))
    ax=plt.subplot(1,2,1)
    early0 = np.hstack(early0)
    late0 = np.hstack(late0)
    early1 = np.hstack(early1)
    late1 = np.hstack(late1)
    plt.scatter(early0*0 + np.random.rand(early0.shape[0])-0.5, 
            early0, color='darkblue', alpha=.5)
    
    # plot also mean as a bar plot
    plt.bar(0, np.nanmean(early0), 0.9, color='darkblue', alpha=.5)

    #
    plt.scatter(late0*0 + np.random.rand(late0.shape[0])-0.5+1,
            late0, color='lightblue', alpha=.5)
    
    # plot also mean as a bar plot
    plt.bar(1, np.nanmean(late0), 0.9, color='lightblue', alpha=.5)

    # compute ks testbetween early0 and late0
    D, p = ks_2samp(early0, late0)
    plt.title(animal_id + " Roi1 early vs. late (KS test): "+str(p))

    #################################################
    ax=plt.subplot(1,2,2)
    plt.scatter(early1*0 + np.random.rand(early1.shape[0])-0.5,
            early1, color='darkred', alpha=.5)
    
    # plot also mean as a bar plot
    plt.bar(0, np.nanmean(early1), 0.9, color='darkred', alpha=.5)

    #
    plt.scatter(late1*0 + np.random.rand(late1.shape[0])-0.5+1,
            late1, color='lightcoral', alpha=.5)
    
    # plot also mean as a bar plot
    plt.bar(1, np.nanmean(late1), 0.9, color='lightcoral', alpha=.5)

    # compute ks testbetween early0 and late0
    D, p = ks_2samp(early1, late1)
    plt.title(animal_id + " Roi2 early vs. late (KS test): "+str(p))

    plt.suptitle(animal_id)
    plt.savefig('/home/cat/'+ animal_id+'_rewards_vs_rois_significance.png')
    plt.show()
    plt.close()







#################################################
#################################################
######### Load upphase data ##
#################################################
animal_ids = [
    # #M1 mice
    # "DON-011733",    # M1    - Processed
    #  'DON-014382',    # M1    - Processed
    #   'DON-014451',    # M1    - Processed
    #   'DON-014575',    # M1    - Processed
    #  "DON-014618",    # M1    - Processed
    #   'DON-015821',    # M1    - Processed
    #   'DON-017923',   # M1    - Not Processed <has extra days at the end....
    #   'DON-018129',   # M1    - Not Processed
    # 'DON-018130',   # M1    - Not Processed

    # #CA3 mice
     "DON-012502",    # CA3    - spreadsheet is old - need reprocessing; ALSO Weird performance, may exclude
     "DON-014266",    # CA3   - Processed
      'DON-014371',    # CA3   - Processed
    'DON-015801',    # CA3   - Processed
      'DON-016658',    # CA3   - Processed
     'DON-019683',    # CA3   - NOT Processed

]

# root_dir = r'F:\bmi\cohort8'
root_dir = '/media/cat/8TB/donato/bmi/'
for animal_id in animal_ids:


    # root_dir = r'F:\bmi\cohort8'
    save_flag = True

    match_rewards_to_rois(root_dir,
                        animal_id,
                        save_flag)

match_array_5mins:  [[9.999999e-01 9.999998e-08]
 [5.000000e-01 5.000000e-01]
 [5.000000e-01 5.000000e-01]
 [9.999998e-08 9.999999e-01]
 [5.000000e-01 5.000000e-01]
 [5.000000e-01 5.000000e-01]]
match_array_5mins:  [[1.0000000e+00 3.3333333e-08]
 [2.5000003e-01 7.5000000e-01]
 [9.9999988e-01 9.9999980e-08]
 [5.0000000e-01 5.0000000e-01]
 [9.9999988e-01 9.9999980e-08]
 [9.9999980e-08 9.9999988e-01]]
match_array_5mins:  [[5.0000000e-01 5.0000000e-01]
 [2.5000000e-08 1.0000000e+00]
 [2.8571430e-01 7.1428573e-01]
 [3.3333334e-01 6.6666669e-01]
 [1.4285715e-08 1.0000000e+00]
 [1.6666669e-01 8.3333331e-01]]
match_array_5mins:  [[1.2500001e-01 8.7500000e-01]
 [1.4285716e-01 8.5714287e-01]
 [6.6666678e-02 9.3333334e-01]
 [2.3076923e-01 7.6923078e-01]
 [9.9999999e-09 1.0000000e+00]
 [1.1111111e-08 1.0000000e+00]]
match_array_5mins:  [[5.0000001e-08 1.0000000e+00]
 [9.0909094e-09 1.0000000e+00]
 [2.0000000e-01 8.0000001e-01]
 [2.2222222e-01 7.7777779e-01]
 [2.2222222e-01 7.7777779e-01]
 [7.69230

In [272]:
####################################################
################## LOAD DATA #######################
####################################################

animal_ids = [
    # #M1 mice
    "DON-011733",    # M1    - Processed
     'DON-014382',    # M1    - Processed
      'DON-014451',    # M1    - Processed
      'DON-014575',    # M1    - Processed
     "DON-014618",    # M1    - Processed
      'DON-015821',    # M1    - Processed
      'DON-017923',   # M1    - Not Processed <has extra days at the end....
      'DON-018129',   # M1    - Not Processed
    'DON-018130',   # M1    - Not Processed

    # #CA3 mice
     "DON-012502",    # CA3    - spreadsheet is old - need reprocessing; ALSO Weird performance, may exclude
     "DON-014266",    # CA3   - Processed
      'DON-014371',    # CA3   - Processed
    'DON-015801',    # CA3   - Processed
      'DON-016658',    # CA3   - Processed
    'DON-019683',    # CA3   - NOT Processed

]

# root_dir = r'F:\bmi\cohort8'
root_dir = '/media/cat/8TB/donato/bmi/'
for animal_id in animal_ids:

# load yaml file for the animal
import yaml
with open(os.path.join(root_dir,
                       animal_id,
                       animal_id+'.yaml')) as file:
    doc = yaml.load(file, Loader=yaml.FullLoader)

try:
    session_ids = np.array(doc['session_names'],dtype='str')
except:
    session_ids = np.array(doc['session_ids'],dtype='str')

#
session_id = session_ids[8]

#

clip_windows = [[0,10],
                #[1050,1100]
                ]

dff_min = 7.5
use_suite2p = True
maximum_std_of_signal = 0.08


#
run_binarization_ensemble(root_dir, 
                          animal_id, 
                          session_id,
                          clip_windows,
                          dff_min,
                          use_suite2p,
                        maximum_std_of_signal)


remove bad cells:  True
...setting 2p parameters...
... computing binarization on non yaml-based data...


binarizing continuous traces : 100%|██████████| 4/4 [00:00<00:00, 296.62it/s]


...saving data...
...saving figures...


In [128]:
# ########################################################
# ############# BINARIZE THE [CA] TRACES #################
# ########################################################

# # double check that we have set the STD thrshold at a reasonable level to catch biggest/highest SNR bursts
# sess.show_plots = True

# #
# sess.visualize_binarization()

In [1]:
# ############################################################
# ############# PLOTS IN HONOR OF CAJAL AND RY ###############
# ############################################################
# sess.show_plots=True
sess.process_session_traces()

NameError: name 'sess' is not defined

In [14]:
############################################################
############# PLOTS IN HONOR OF CAJAL AND RY ###############
############################################################

#
#sess.load_calibration()

# run this if you have run the calibration through the bmi gui
#sess.load_calibration_new()

#
sess.show_plots=True
sess.show_session_traces_and_calibration()

#F:\bmi\cohort8\DON-014451\20230328\calibration
#F:\bmi\cohort8\DON-014451\20230328\\results.npz'

AttributeError: 'ProcessSession' object has no attribute 'e1_1'

In [5]:
#################################################################
############# CORREALTION BETWEEN ENSEMBLES #####################
#################################################################
# this is not so interesting; may need to do more on this
sess.show_plots = True

#
sess.compute_ensemble_correlations()


pearson correlation E1 cells;  0.03916222897242536
pearson correlation E2 cells;  0.09556712722981311


In [6]:
#################################################################
############# CORRELOGRAMS BETWEEN REWARDS AND LICKING ##########
#################################################################
# this is not so interesting; may need to do more on this
sess.show_plots = True
#
sess.window_size = 60   # seconds
sess.compute_correlograms_reward_vs_licking()


lick times:  [   3.18    3.19    3.2  ... 2985.97 2985.98 2985.99]
# of spikes:  12406


model filter: remove bleaching or trends: 100%|██████████| 4/4 [00:00<00:00, 1789.19it/s]
binarizing continuous traces filtered fluorescence onphase: 100%|██████████| 4/4 [00:00<00:00, 434.89it/s]
binarizing continuous traces filtered fluorescence upphase: 100%|██████████| 4/4 [00:00<00:00, 172.19it/s]


In [20]:
###################################################################################
############COMPUTE CORRELOGRAMS BETWEEN CELLS BASED ON UPPHASE DETECTION #########
###################################################################################
#
sess.corr_window = 60   # in seconds
sess.compute_correlograms_ensembles_upphase()

In [2]:
###########################################################################################
############COMPUTE CORRELOGRAMS BETWEEN CELLS BASED ON RAW FILTERED FLUORESCENCE #########
##########################################################################################
#
sess.show_plots = True
sess.window = 60*30   # in seconds
sess.compute_correlograms_ensembles_fluorescence()

NameError: name 'sess' is not defined

In [11]:
########################################################
####### COMPUTE INTRA-SESSION REWARD HISTOGRAM #########
########################################################

#
sess.show_plots=True
sess.bin_width = 5 # in minutes
sess.compute_intra_session_reward_histogram()



Perason corr:  PearsonRResult(statistic=0.3706110830816283, pvalue=0.2917649089674079)


In [12]:
######################################################################
####### COMPUTE INTRA-SESSION REWARD HISTOGRAM - HITS/MINUTE #########
######################################################################

#
sess.show_plots=True
sess.bin_width_mins = 5 # in minutes
sess.compute_intra_session_reward_histogram_hist_per_minute()

Perason corr:  PearsonRResult(statistic=0.37154698242568246, pvalue=0.2904522471458872)


In [13]:
##########################################################################
####### COMPUTE INTRA-SESSION CELL BURST HISTOGRAMS AS LINE PLOT #########
##########################################################################

#
sess.show_plots=True
sess.bin_width = 5 # in minutes
sess.compute_intra_session_cell_burst_histogram()


In [14]:
##########################################################################
####### COMPUTE INTRA-SESSION CELL BURST HISTOGRAMS AS LINE PLOT #########
##########################################################################
#
sess.compute_intra_session_cell_burst_histogram_v2()

In [7]:
####################################################
####### INTER-BURST-INTERVAL DISTRIBUTIONS #########
####################################################
#
sess.isi_width = 10
sess.isi_bin_width = 0.250
sess.compute_intra_session_inter_burst_interval()

In [67]:
#######################################
####### TIME TO REWARD PER TRIAL ######
#######################################

#
sess.time_to_reward_per_trial_intra_session()



nan


In [161]:
###########################################
####### BINNED PAIRWISE CORRELATIONS ######
###########################################

#
sess.session_id = '20230202'
#sess.session_id = '20230131'
sess.pairwise_correlations_intra_session_dff()



In [207]:
##################################
####### LICKS VS. ENSEMBLES ######
##################################

#
sess.session_id = '20230202'
#sess.session_id = '20230131'
sess.load_data()

# not super informative
#sess.compute_licks_vs_ensembles()


#
sess.window =30*30  # 30 seconds
sess.compute_licks_vs_ensembles_correlograms()


missing:  abs_times_ttl_read
missing:  abs_times_ca_read
matching lick times to ttl Times:  (89955,)


100%|██████████| 89955/89955 [00:35<00:00, 2537.89it/s] 


reward times:  (31,)
Recording length (mins):  50.0
abs times:  (3008338,) [ 149.8581727  149.8583522  149.8585126 ... 3156.1436586 3156.1453174
 3156.1455295]
ttl times:  (89955,) 0.0 2999.9520262  total rec time sec:  2999.9520262
ttl computed:  (89955,) [    0     1     2 ... 89997 89998 89999]
ttl detected:  (89955,) [    0     1     2 ... 89952 89953 89954]
lick detector:  (3008338, 1)
lick times:  [ 587.9077456  587.9079715  587.9097345 ... 2703.8404269 2703.8421129
 2703.8424087]
E1 , E2,  (2, 90000) (2, 90000)
(89956,) (89956,)


100%|██████████| 1800/1800 [00:04<00:00, 416.59it/s]


In [199]:
d = np.load('/media/cat/4TB/donato/andres/DON-011733/20230202/data/lick_times_ttl.npy')
print (d.shape)

(89956,)


In [160]:
# ###################################################
# ######## LOAD ROI CONTOURS AND VISUALIZE ##########
# ###################################################

# data = np.load('/media/cat/4TB/donato/andres/DON-011733/20230127/data/results.npz')
# rois_e1 = data['rois_pixels_ensemble1'].T
# rois_e2 = data['rois_pixels_ensemble2'].T
# print (rois_e1.shape)
# plt.figure()

# plt.scatter(rois_e1[:,0], rois_e1[:,1])
# plt.scatter(rois_e2[:,0], rois_e2[:,1])
# plt.xlim(0,512)
# plt.ylim(0,512)
# plt.show()
# # #

(490, 2)


In [22]:
####################################################
################ SESSION % HITS ####################
####################################################

root_dir = '/media/cat/4TB/donato/andres/Cohort 6 Data/'
animal_id = 'DON-014266'
session_ids = [
    #'20230127',
    #'20230128',
    #'20230129',
    #'20230130',
    '20230131',
    '20230201',
    '20230202',
    '20230203',
    '20230204',
    '20230205',
    '20230206',
    #'20230207',

]

# define the 3 sets of sessions for contingency degradation
# leave blank for non-used
t = ['20230131',
        '20230201']
cd = ['20230202',
        '20230203']
r = ['20230205',
        '20230206']


# define early vs. late sessions
early = ['20230131',
        '20230201',
        '20230202',
        ]
late = ['20230204',
        '20230205',
        '20230206',
       ]

In [23]:
#####################################################
#####################################################
#####################################################
      
sess = ProcessSession(root_dir,
                      animal_id,
                     session_ids[0])

sess.load_data()

#
sess.bin_width_mins = 5
sess.verbose=False
sess.session_ids = session_ids

sess.compute_animal_hit_rate()

In [17]:
import numpy as np
fname = '/media/cat/8TB/donato/bmi/DON-014618/20230505/results.npz'
data = np.load(fname, allow_pickle=True)    

abs_times = data['abs_times_ttl_read']
ttl_times = data['ttl_times']

lick_detector = data['lick_detector_abstime']
idx = np.where(lick_detector > 3)[0]
lick_times = abs_times[idx] - ttl_times[0]
print (lick_times)

dd = data['abs_times_ca_read']
print (dd.shape)


[2.43144700e-01 2.43368100e-01 2.43546400e-01 ... 2.86109859e+03
 2.86109895e+03 2.86110055e+03]
(90000,)


In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import yaml
%matplotlib tk


animal_ids = [
    # #M1 mice
    "DON-011733",    # M1    - Processed
     'DON-014382',    # M1    - Processed
     'DON-014451',    # M1    - Processed
    'DON-014575',    # M1    - Processed
    "DON-014618",    # M1    - Processed
     'DON-015821',    # M1    - Processed
    'DON-017923',   # M1    - Not Processed <has extra days at the end....
    'DON-018129',   # M1    - Not Processed
    'DON-018130',   # M1    - Not Processed

    # #CA3 mice
    "DON-012502",    # CA3    - spreadsheet is old - need reprocessing; ALSO Weird performance, may exclude
    "DON-014266",    # CA3   - Processed
    'DON-014371',    # CA3   - Processed
    'DON-015801',    # CA3   - Processed
    'DON-016658',    # CA3   - Processed
    'DON-019683'
]

root_dir = '/media/cat/8TB/donato/bmi/'

for animal_id in animal_ids:
    # load yaml file from this location
    fname = os.path.join(root_dir, animal_id, 
                        str(animal_id)+'.yaml')
    yam = yaml.load(open(fname, 'r'), Loader=yaml.FullLoader)
    # grab "session_ids" from yaml file
    sessions = yam['session_ids']



    # make color map over 7 discrete viridis
    cmap = plt.cm.viridis
    colors = cmap(np.linspace(0, 1, len(sessions)-1))
    cell_names = ['pos1', 'pos2', 'neg1', 'neg2']
    #
    plt.figure(figsize=(8,8))
    cell_array = [[],[],[],[]]
    for ctr,session in enumerate(sessions[1:]):
        fname = os.path.join(root_dir,
                            animal_id, 
                            str(session), 
                            'results',
                            'cell_burst_histogram_v2.npy')
        data = np.load(fname, allow_pickle=True)
        print (data.shape)
        t = np.arange(6)*5+2.5
        for k in range(data.shape[0]):
            ax=plt.subplot(2,2,k+1)
            plt.plot(t,
                    data[k],
                    color=colors[ctr],
                    linewidth=2,
                    )
            
            plt.title("Cell: "+str(k))
            cell_array[k].append(data[k])

    # plot also average of all cells
    for k in range(4):
        ax=plt.subplot(2,2,k+1)
        temp = np.array(cell_array[k])
        std = temp.std(0)
        temp = temp.mean(0)
        plt.ylabel("burst rate(5min bins)")
        plt.plot(t,
                temp, 
                color='black',
                linewidth=2)
        
        #
        plt.fill_between(t,
                        temp-std,
                        temp+std,
                        color='black',
                        alpha=0.2)
        plt.title("Cell: "+str(cell_names[k]))

    plt.suptitle("Cell burst histograms " + animal_id)
    plt.show()

    # 
    plt.savefig('/home/cat/'+animal_id+'_cell_burst_histograms.png')

(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
(4, 6)
